# incrusive
## Advantages
1. Recursive functions make the code look clean and elegant.
2. A complex task can be broken down into simpler sub-problems using recursion.
3. Sequence generation is easier with recursion than using some nested iteration.
## weakness
1. Sometimes the logic behind recursion is hard to follow through.
2. Recursive calls are expensive (inefficient) as they take up a lot of memory and time.
3. Complex recursive functions are hard to debug.
### features
1: recursion Direction:  start from most complicated input to simplest input
2. easy to get result with the simplest input 
3. easy to go into the next input given a current input. 
4. Not difficult to get relationships between the results of a current input and its next input.

Flow control: suppose there are n inputs. always get the next input and output based on current input and output
func(n)->func(n-1)->...->func(1)

In [ ]:
recursive patthern

def func(...):
    #base case: result with simplest input
    #Check to see whether the current value(s) being processed match the base case.
    if ...:
        return ...
    #recursion
    #Redefine the answer in terms of a smaller or simpler sub-problem or sub-problems.
    else:
        #Run the algorithm on the sub-problem.
        retrun func(...)

# create _search for elasticsearch

In [116]:
import re
def string_to_dict(input_str):
    input_str = re.sub(r"_or=|_and=", "", input_str)
    str_list = re.findall(r"[\w\.]*=[\w\.]*", input_str)
    str_pair = dict([ i.split("=") for i in str_list])
    return str_pair
#
print(string_to_dict("a=d"))
print(string_to_dict("a=d,b.c=343"))
print(string_to_dict("(a=d,b.c=343)"))
print(string_to_dict("_or=(a=d,b.c=343)"))
print(string_to_dict("_or=a=d,b.c=343"))
print(string_to_dict("_and=(a=d,b.c=343)"))
print(string_to_dict("_and=a=d,b.c=343"))

{'a': 'd'}
{'a': 'd', 'b.c': '343'}
{'a': 'd', 'b.c': '343'}
{'a': 'd', 'b.c': '343'}
{'a': 'd', 'b.c': '343'}
{'a': 'd', 'b.c': '343'}
{'a': 'd', 'b.c': '343'}


In [220]:
import re
def split_boolean_string(input_str):
    if "(" in input_str:
        or_list=[]
        start, acc = -1, 0
        for x,y in enumerate(input_str):
            if y=="(":
                if start==-1:
                    start = x
                else:
                    acc += 1
            elif y==")":
                if acc ==0:
                    or_list.append(input_str[start+1:x])
                    start, acc =-1, 0
                else:
                    acc -= 1
        return or_list
    return [input_str]

input_str="(_and=A=a,B=b),(_and=(_or=(_and=1=1,2=2),(_and=3=3,4=4))),(_or=C=c,D=d,E.id=e)"
or_list = split_boolean_string(input_str)
print(or_list)

input_str="(_and=A=a,B=b),(_or=C=c,D=d,E.id=e)"
or_list = split_boolean_string(input_str)
print(or_list)

input_str="(A=a,B=b)"
or_list = split_boolean_string(input_str)
print(or_list)

input_str="A=a,B=b"
or_list = split_boolean_string(input_str)
print(or_list)

input_str="_or=(A=a),(B=b),(_and=C=c,D=d)"
or_list = split_boolean_string(input_str)
print(or_list)

['_and=A=a,B=b', '_and=(_or=(_and=1=1,2=2),(_and=3=3,4=4))', '_or=C=c,D=d,E.id=e']
['_and=A=a,B=b', '_or=C=c,D=d,E.id=e']
['A=a,B=b']
['A=a,B=b']
['A=a', 'B=b', '_and=C=c,D=d']


In [54]:
#create bool-must query    
def create_and_must(input_str):
    str_dict = string_to_dict(input_str)
    bool_dict = {
        "bool":{"must":[]}
    }
    for k,v in str_dict.items():
        bool_dict["bool"]["must"].append({"match":{k:v}})
    return bool_dict

#
print(create_and_must("a=d,b.c=343"))

{'bool': {'must': [{'match': {'a': 'd'}}, {'match': {'b.c': '343'}}]}}


In [224]:
#create bool-should query    
def create_or_should(input_str):
    str_dict = string_to_dict(input_str)
    bool_dict = {"bool":{"should":[]}}
    if len(str_dict.keys())>1:
        bool_dict["bool"]["minimum_should_match"] = 1
    for k,v in str_dict.items():
        bool_dict["bool"]["should"].append({"match":{k:v}})
    return bool_dict

#
print(create_or_should("a=d,b.c=343"))

{'bool': {'should': [{'match': {'a': 'd'}}, {'match': {'b.c': '343'}}], 'minimum_should_match': 1}}


In [182]:
#convert string to json
import json
import re
s1 = "(_or=A=a,B.id=b),(_and=C,D)"
print(set(re.findall(r"_or=|_and=", s1)))
s1 = "(_or=A=a,B.id=b)"
print(set(re.findall(r"_or=|_and=", s1)))



{'_or=', '_and='}
{'_or='}


In [244]:

def create_query(myStr, parent_bool, bool_type=None):
    empty_bool = {"bool":{"must":[],"should":[]}}
    boolean_list = split_boolean_string(myStr)
    
    if len(boolean_list)>1:
        #print("###", boolean_list)
        parent_bool["bool"]["minimum_should_match"] = len(boolean_list) if bool_type == '_and' else 1

    for boolean_str in boolean_list:
        boolean_counts = set(re.findall(r"_or=|_and=", boolean_str))
        #print('==incursive:', boolean_str)        
        if boolean_counts == {"_and=", "_or="}:
            #print('==incursive:', boolean_str)
            if boolean_str.startswith('_and'):
                parent_bool["bool"]["should"].append(create_query(boolean_str, empty_bool, '_and'))    
            else:
                parent_bool["bool"]["should"].append(create_query(boolean_str, empty_bool))
        elif boolean_counts == {"_and="}:
            bool_dict = create_and_must(boolean_str)
            parent_bool["bool"]["should"].append(bool_dict)
        else:
            bool_dict = create_or_should(boolean_str)
            parent_bool["bool"]["should"].append(bool_dict)
    #print(parent_bool,"\n")
    return parent_bool

#
input_str="C=c,D=d,E.id=e"
query_bool = create_query(input_str, {"bool":{"must":[],"should":[]}})
print(query_bool, end="\n\n")

#
input_str="_or=(_and=A=a,B=b),(_or=C=c,D=d,E.id=e),(_and=(_or=(_and=1=1,2=2),(_and=3=3,4=4)))"
query_bool = create_query(input_str, {"bool":{"must":[],"should":[]}})
print(query_bool, end="\n\n")

input_str="(_or=(C=c),(D=d),(_and=(E.id=e, F=f)))"
query_bool = create_query(input_str, {"bool":{"must":[],"should":[]}})
print(query_bool, end="\n\n")

input_str="(_or=(C=c),(_and=(_or=D=d,B=b),(A=a)),(_and=(E.id=e, F=f)))"
query_bool = create_query(input_str, {"bool":{"must":[],"should":[]}})
print(query_bool, end="\n\n")

{'bool': {'must': [], 'should': [{'bool': {'should': [{'match': {'C': 'c'}}, {'match': {'D': 'd'}}, {'match': {'E.id': 'e'}}], 'minimum_should_match': 1}}]}}

{'bool': {'must': [], 'should': [{'bool': {'must': [{'match': {'A': 'a'}}, {'match': {'B': 'b'}}]}}, {'bool': {'should': [{'match': {'C': 'c'}}, {'match': {'D': 'd'}}, {'match': {'E.id': 'e'}}], 'minimum_should_match': 1}}, {'bool': {'must': [], 'should': [{'bool': {'must': [], 'should': [{'bool': {'must': [{'match': {'1': '1'}}, {'match': {'2': '2'}}]}}, {'bool': {'must': [{'match': {'3': '3'}}, {'match': {'4': '4'}}]}}], 'minimum_should_match': 1}}]}}], 'minimum_should_match': 1}}

{'bool': {'must': [], 'should': [{'bool': {'must': [], 'should': [{'bool': {'should': [{'match': {'C': 'c'}}]}}, {'bool': {'should': [{'match': {'D': 'd'}}]}}, {'bool': {'must': [{'match': {'E.id': 'e'}}, {'match': {'F': 'f'}}]}}], 'minimum_should_match': 1}}]}}

{'bool': {'must': [], 'should': [{'bool': {'must': [], 'should': [{'bool': {'should': [{

### DNA sequence
ATGC
-1 C->G
-2 G->C
-3 T->A
-4 A->T

In [88]:
def print_str(instr):
    if len(instr)==0:
        return ''
    else:
        print(instr[0])
        return instr[0]+print_str(instr[1:])
print_str('sadfsdaf')

s
a
d
f
s
d
a
f


'sadfsdaf'

In [94]:
def reverse_seq(dnaseq):
    return dnaseq[::-1]

def com_nt(nt):
    c= {'A':'T', 'T':'A','G':'C','C':'G'}
    if nt in c:
        return c[nt]
    return 'N'
    
def rev_com(dnaseq):
    #print(dnaseq)
    if len(dnaseq)==0:
        return ''
    else:
        return com_nt(dnaseq[-1]) + rev_com(dnaseq[:-1])

print(rev_com('ATGCGAXCATTCG'))

CGAATGNTCGCAT


In [98]:
#translate DNA to protein
def trans_aa(dna):
    aa_dict = {'ATG': "M", 'TCC':'L', 'TCG':'D'}
    if dna in aa_dict:
        return aa_dict[dna]
    return '.'

def trans_dna(dna):
    if len(dna)==3:
        return trans_aa(dna)
    elif len(dna)<3:
        return '.'
    else:
        return trans_aa(dna[:3]) + trans_dna(dna[3:])
print(trans_dna('ATGTCCTCG'))
print(trans_dna('ATGTCCTC'))

MLD
ML.


In [ ]:
class TreeNode:
    def __init__(self):
        self.children = {}
class Trie:
    def __init__(self):
        self.root = TreeNode()

In [598]:
int(0b110)

6

In [595]:
x=24
print(bin(x), x&1)

0b11000 0


In [599]:
a,b=3,9
print(bin(a), bin(b), bin(a^b))

0b11 0b1001 0b1010


In [600]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None, next=None):
        self.value = val
        self.left = left
        self.right = right
        self.next = next

class Solution:
    

In [442]:
ord('a')-ord('b')

-1

In [602]:
a = [][]

SyntaxError: invalid syntax (<ipython-input-602-865e0b921b43>, line 1)